In [ ]:
import nannyml as nml
from IPython.display import display

reference, analysis, analysis_targets = nml.load_synthetic_car_loan_dataset()
display(reference.head())

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(reference.head())

In [ ]:
selected_columns = [
    'car_value', 'debt_to_income_ratio', 'driver_tenure'
]
calc = nml.SummaryStatsMedianCalculator(
    column_names=selected_columns,
)

In [ ]:
calc.fit(reference)
results = calc.calculate(analysis)
display(results.filter(period='all').to_df())

In [ ]:
for column_name in results.column_names:
    results.filter(column_names=column_name).plot().show()

In [ ]:
calc1 = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['roc_auc', 'f1', 'precision', 'recall', 'specificity', 'accuracy'],
    chunk_size=5000)

calc1.fit(reference)

analysis = analysis.merge(analysis_targets, left_index=True, right_index=True)
results1 = calc1.calculate(analysis)

ranker1 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker1.fit(
    results1.filter(period='reference', metrics=['roc_auc']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features1 = ranker1.rank(
    results,
    results1.filter(metrics=['roc_auc']),
    only_drifting = False)

display(correlation_ranked_features1)